In [55]:
from __future__ import print_function
from __future__ import unicode_literals
import yaml
import procesamiento_microdatos as pm

In [56]:
pan = pm.generate_panel()

In [76]:
with open("grupos.yaml") as groups_file:
    grupos = yaml.load(groups_file)

In [77]:
grupos

[{u'Energ\xedas Primarias': [u'Energ\xeda Hidr\xe1ulica',
   u'Energ\xeda Nuclear',
   'Gas Natural de Pozo',
   u'Petr\xf3leo',
   {u'Otras Fuentes Primarias de Energ\xeda': [u'Energ\xeda E\xf3lica',
     u'Energ\xeda Solar',
     u'Carb\xf3n Mineral',
     {'Aceites y Alcoholes Vegetales': ['Alcohol Vegetal',
       'Aceites Vegetales']},
     {u'Le\xf1a, Bagazo y Otros': [u'Le\xf1a',
       'Bagazo',
       'Otros Primarios']}]}]},
 {u'Energ\xedas Secundarias': [u'Energ\xeda El\xe9ctrica',
   'Gas Distribuido por Redes',
   'Gas Licuado',
   {u'Derivados del Petr\xf3leo': ['Gas Oil', 'Fuel Oil', 'Motonaftas']},
   {'Biocombustibles': ['Biodiesel', 'Bioetanol']},
   {u'Otras Fuentes Secundarias de Energ\xeda': ['Gas de Alto Horno',
     u'No Energ\xe9tico de Carb\xf3n',
     u'Carb\xf3n Vegetal',
     {u'Energ\xedas Secundarias del Petr\xf3leo': [u'Carb\xf3n Residual',
       u'Coque de Petr\xf3leo',
       u'Gas de Refiner\xeda',
       'Kerosene',
       u'No Energ\xe9tico',
      

In [60]:
def flatten_tree(tree):
    if isinstance(tree, (unicode, str)):
        print(tree)
    elif isinstance(tree, list):
        
                  for node in tree:
        print("{}: {}".format(node, type(node)))

In [78]:
flatten_tree(grupos)

{u'Energ\xedas Primarias': [u'Energ\xeda Hidr\xe1ulica', u'Energ\xeda Nuclear', 'Gas Natural de Pozo', u'Petr\xf3leo', {u'Otras Fuentes Primarias de Energ\xeda': [u'Energ\xeda E\xf3lica', u'Energ\xeda Solar', u'Carb\xf3n Mineral', {'Aceites y Alcoholes Vegetales': ['Alcohol Vegetal', 'Aceites Vegetales']}, {u'Le\xf1a, Bagazo y Otros': [u'Le\xf1a', 'Bagazo', 'Otros Primarios']}]}]}: <type 'dict'>
{u'Energ\xedas Secundarias': [u'Energ\xeda El\xe9ctrica', 'Gas Distribuido por Redes', 'Gas Licuado', {u'Derivados del Petr\xf3leo': ['Gas Oil', 'Fuel Oil', 'Motonaftas']}, {'Biocombustibles': ['Biodiesel', 'Bioetanol']}, {u'Otras Fuentes Secundarias de Energ\xeda': ['Gas de Alto Horno', u'No Energ\xe9tico de Carb\xf3n', u'Carb\xf3n Vegetal', {u'Energ\xedas Secundarias del Petr\xf3leo': [u'Carb\xf3n Residual', u'Coque de Petr\xf3leo', u'Gas de Refiner\xeda', 'Kerosene', u'No Energ\xe9tico', 'Otras Naftas']}, {u'Energ\xedas Secundarias del Carb\xf3n': [u'Coque de Carb\xf3n', u'Gas de Coquer\xeda

In [38]:
grupos["Energías Primarias"]

[u'Energ\xeda Hidr\xe1ulica',
 u'Energ\xeda Nuclear',
 'Gas Natural de Pozo',
 u'Petr\xf3leo',
 {u'Otras Fuentes Primarias de Energ\xeda': [u'Energ\xeda E\xf3lica',
   u'Energ\xeda Solar',
   u'Carb\xf3n Mineral',
   {'Aceites y Alcoholes Vegetales': ['Alcohol Vegetal', 'Aceites Vegetales']},
   {u'Le\xf1a, Bagazo y Otros': [u'Le\xf1a', 'Bagazo', 'Otros Primarios']}]}]

In [69]:
ep = grupos["Energías Primarias"]

In [79]:
def flatten(coso, flat_list=[]):
    for elem in coso:
        if isinstance(elem, (unicode, str)):
            flat_list.append(elem)
        else:
            for key in elem:
                flat_list.append(key)
                flatten(elem[key], flat_list)
    return flat_list

In [80]:
flatten(grupos)

[u'Energ\xedas Primarias',
 u'Energ\xeda Hidr\xe1ulica',
 u'Energ\xeda Nuclear',
 'Gas Natural de Pozo',
 u'Petr\xf3leo',
 u'Otras Fuentes Primarias de Energ\xeda',
 u'Energ\xeda E\xf3lica',
 u'Energ\xeda Solar',
 u'Carb\xf3n Mineral',
 'Aceites y Alcoholes Vegetales',
 'Alcohol Vegetal',
 'Aceites Vegetales',
 u'Le\xf1a, Bagazo y Otros',
 u'Le\xf1a',
 'Bagazo',
 'Otros Primarios',
 u'Energ\xedas Secundarias',
 u'Energ\xeda El\xe9ctrica',
 'Gas Distribuido por Redes',
 'Gas Licuado',
 u'Derivados del Petr\xf3leo',
 'Gas Oil',
 'Fuel Oil',
 'Motonaftas',
 'Biocombustibles',
 'Biodiesel',
 'Bioetanol',
 u'Otras Fuentes Secundarias de Energ\xeda',
 'Gas de Alto Horno',
 u'No Energ\xe9tico de Carb\xf3n',
 u'Carb\xf3n Vegetal',
 u'Energ\xedas Secundarias del Petr\xf3leo',
 u'Carb\xf3n Residual',
 u'Coque de Petr\xf3leo',
 u'Gas de Refiner\xeda',
 'Kerosene',
 u'No Energ\xe9tico',
 'Otras Naftas',
 u'Energ\xedas Secundarias del Carb\xf3n',
 u'Coque de Carb\xf3n',
 u'Gas de Coquer\xeda',
 u'E